In [37]:
import os
from pathlib import Path

import pandas as pd

In [38]:
INPUT_COL = 'Processed comment_text'
OUTPUT_COL = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

CWD = Path(os.getcwd()).parent.parent
DATA_DIR = Path('data/preprocessed/jigsaw_toxic_comments')

In [39]:
DATA_DIR = CWD / DATA_DIR

train_data = pd.read_csv(DATA_DIR / 'train.csv')
validation_data = pd.read_csv(DATA_DIR / 'test.csv')

In [40]:
# Count of NA values in Processed comment
train_data[train_data.isnull().any(axis=1)].head()

,id,comment_text,Processed comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
2098,05ac7a7a83e4c63a,"No, it doesn´t.80.228.65.162",NaN,0,0,0,0,0,0
2407,067638a445ccd93b,"Here, here and here.",NaN,0,0,0,0,0,0
3990,0aa6f3529219b37e,From here\n\nFrom here 160.80.2.8,NaN,0,0,0,0,0,0
4482,0bed2196c873636d,1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n...,NaN,0,0,0,0,0,0
6063,10304d5db5fb5821,n i g g e r f a g g o t,NaN,1,1,0,0,0,1


In [41]:
train_data = train_data.dropna(subset=['Processed comment_text'])
train_data.head()

,id,comment_text,Processed comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,daww match background colour seemingli stuck t...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",hey man realli tri edit war guy constantli rem...,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",cant make real suggest improv wonder section s...,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",sir hero chanc rememb page that,0,0,0,0,0,0


In [42]:
def test_train(test_size=0.2):
    from sklearn.model_selection import train_test_split

    return train_test_split(
        train_data[INPUT_COL].values,
        train_data[OUTPUT_COL],
        test_size=test_size
    )

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = test_train()

tfidf_vectorizer = TfidfVectorizer() 
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

models = {}
for col in y_train.columns:
    model = LogisticRegression(max_iter=250)
    model.fit(X_train, y_train[col])
    models[col] = model

for col in y_train.columns:
    print(f'{col}: {model[col].score(X_test, y_test)}')

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = test_train()
count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(X_train)
X_test = count_vectorizer.transform(X_test)

alpha_ranges = {
    "alpha": [0.001, 0.01, 0.1, 1, 10.0, 100]
}

mnb = MultinomialNB(alpha=1)
mnb = mnb.fit(X_train, y_train)
mnb.score(X_test, y_test)

0.9507775269626285